In [92]:
import requests
import pandas as pd

In [2]:
# Esta funcion imprime las 10 monedas volatiles con mas capitalizacion y luego las 10 monedas estables.
def get_top_coins():
    stable = []
    volatil = []

    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()

    for coin in data:
        if coin['current_price'] > 1 or 0.99 > coin['current_price'] and len(volatil) < 10:
            volatil.append(coin['id'])
        elif 1.1 > coin['current_price'] > 0.97 and len(stable) < 10:
            stable.append(coin['id'])
        else:
            break
        
    return stable,volatil

stable, volatil = get_top_coins()
print(stable)
print(volatil)

['tether', 'usd-coin']
['bitcoin', 'ethereum', 'binancecoin', 'ripple', 'staked-ether', 'cardano', 'dogecoin', 'solana', 'tron', 'polkadot']


In [185]:
# Utiliza una lista de nombres de criptomonedas y a traves de la api de coingecko extrae la informacion de cada una de ellas para luego de un ETL guardar 
# la informacion en un archivo csv.
def get_historical_data(coinnamelist,days):
    for coin in coinnamelist:
        url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart"
        params = {
            'vs_currency': 'usd',
            'days': days,
            'interval': 'daily'
        }
    
        response = requests.get(url, params=params)
    
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)
            df['market_caps'] = df['market_caps'].apply(lambda x: int(x[1]))
            df['total_volumes'] = df['total_volumes'].apply(lambda x: int(x[1]))
            df['Date'] = df['prices'].apply(lambda x: pd.to_datetime(x[0], unit='ms'))
            df['prices'] = df['prices'].apply(lambda x: int(x[1]))
            df.rename(columns={'prices': 'Price_usd','total_volumes':'Volume','market_caps':'Market_cap'}, inplace=True)
            df = df.groupby(df["Date"].dt.to_period("M")).tail(1)
            df.to_csv(f'./Data/{coin}.csv',index=False)
        else:
            print(f"Error al obtener los datos de {coin}:", response.status_code)


In [28]:
get_historical_data(volatil)

***

Obtencion de datos globales

In [3]:
url = 'https://api.coingecko.com/api/v3/global'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

In [4]:
data

{'data': {'active_cryptocurrencies': 10150,
  'upcoming_icos': 0,
  'ongoing_icos': 49,
  'ended_icos': 3376,
  'markets': 836,
  'total_market_cap': {'btc': 41909168.0563866,
   'eth': 659824072.8019023,
   'ltc': 16787531520.292736,
   'bch': 5725551987.714669,
   'bnb': 5107265646.735155,
   'eos': 1869087785530.507,
   'xrp': 2084736384445.723,
   'xlm': 8719990169319.429,
   'link': 174022950370.4369,
   'dot': 247451527525.28854,
   'yfi': 196650428.46162176,
   'usd': 1111275380341.9402,
   'aed': 4081753366634.2646,
   'ars': 388900709701547.44,
   'aud': 1715246881905.506,
   'bdt': 121618100976189.36,
   'bhd': 418990824302.6044,
   'bmd': 1111275380341.9402,
   'brl': 5400909475999.873,
   'cad': 1502434312743.8823,
   'chf': 975814245304.3989,
   'clp': 958819510912830.6,
   'cny': 7986736158517.532,
   'czk': 24635820567060.652,
   'dkk': 7622962385313.375,
   'eur': 1022662281513.4741,
   'gbp': 872999047115.1628,
   'hkd': 8713454693492.135,
   'huf': 391243091509044.3,


In [5]:
data = data['data']
df_global = pd.DataFrame(data)
df_global.head()

,active_cryptocurrencies,upcoming_icos,ongoing_icos,ended_icos,markets,total_market_cap,total_volume,market_cap_percentage,market_cap_change_percentage_24h_usd,updated_at
btc,10150,0,49,3376,836,4.190917e+07,1.946119e+06,46.447861,2.646764,1692812860
eth,10150,0,49,3376,836,6.598241e+08,3.063998e+07,18.243934,2.646764,1692812860
ltc,10150,0,49,3376,836,1.678753e+10,7.795558e+08,NaN,2.646764,1692812860
bch,10150,0,49,3376,836,5.725552e+09,2.658751e+08,NaN,2.646764,1692812860
bnb,10150,0,49,3376,836,5.107266e+09,2.371640e+08,3.017118,2.646764,1692812860


In [150]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, btc to sol
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   active_cryptocurrencies               67 non-null     int64  
 1   upcoming_icos                         67 non-null     int64  
 2   ongoing_icos                          67 non-null     int64  
 3   ended_icos                            67 non-null     int64  
 4   markets                               67 non-null     int64  
 5   total_market_cap                      61 non-null     float64
 6   total_volume                          61 non-null     float64
 7   market_cap_percentage                 10 non-null     float64
 8   market_cap_change_percentage_24h_usd  67 non-null     float64
 9   updated_at                            67 non-null     int64  
dtypes: float64(4), int64(6)
memory usage: 5.8+ KB


Faltan datos importante en las columnas total_market_cap y market_cap_percentage

In [6]:
df_global['market_cap_percentage'].sum()

83.74989519178826

La sumatoria de la columna market_cap_percentage (porcentaje de capitalizacion) no llega al 100 % o cercano.

In [12]:
df_global = df_global.sort_values(by='market_cap_percentage', ascending=False)
df_global.head()

AttributeError: 'list' object has no attribute 'sort_values'

***

Utilizando la opcion /coins/market de la api de coingecko para obtener datos mas completos.

In [145]:
# Esta funcion retorna los datos de todas las criptomonedas en coingecko
def all_coins():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 250,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()
        
    return data

In [146]:
df_global = all_coins()

In [147]:
df_global = pd.DataFrame(df_global)
df_global.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,25954.000000,505136337605,1,5.448962e+11,1.018509e+10,26200.000000,...,2.100000e+07,2.100000e+07,69045.00,-62.42162,2021-11-10T14:24:11.849Z,67.810000,38163.21344,2013-07-06T00:00:00.000Z,None,2023-08-25T19:46:17.863Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1645.340000,197735652054,2,1.977357e+11,6.235384e+09,1664.660000,...,1.202164e+08,NaN,4878.26,-66.28221,2021-11-10T14:24:19.604Z,0.432979,379789.61182,2015-10-20T00:00:00.000Z,"{'times': 83.76551925582277, 'currency': 'btc'...",2023-08-25T19:46:18.945Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999135,82741773321,3,8.274177e+10,1.857318e+10,1.002000,...,8.284913e+10,NaN,1.32,-24.51760,2018-07-24T00:00:00.000Z,0.572521,74.43974,2015-03-02T00:00:00.000Z,None,2023-08-25T19:45:00.424Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,217.120000,33413255437,4,4.343441e+10,3.977366e+08,219.050000,...,1.538562e+08,2.000000e+08,686.31,-68.35641,2021-05-10T07:24:17.097Z,0.039818,545315.89525,2017-10-19T00:00:00.000Z,None,2023-08-25T19:46:18.062Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.524894,27769990717,5,5.248118e+10,9.220591e+08,0.529816,...,9.998849e+10,1.000000e+11,3.40,-84.56456,2018-01-07T00:00:00.000Z,0.002686,19428.12541,2014-05-22T00:00:00.000Z,None,2023-08-25T19:46:13.355Z


In [144]:
df_global

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,nano,xno,Nano,https://assets.coingecko.com/coins/images/756/...,0.657915,87676032,257,8.767603e+07,19002156.0,0.845856,...,1.332483e+08,1.332483e+08,33.690000,-98.04435,2018-01-02T00:00:00.000Z,0.026179,2417.02419,2017-07-16T00:00:00.000Z,None,2023-08-25T19:41:07.114Z
1,blox,cdt,Blox,https://assets.coingecko.com/coins/images/1231...,0.129246,87271059,251,1.293711e+08,123279.0,0.174764,...,1.000000e+09,NaN,0.502216,-74.23992,2023-03-21T14:15:03.845Z,0.001648,7750.75001,2020-03-13T02:24:15.284Z,"{'times': -0.5212144173391808, 'currency': 'et...",2023-08-25T19:41:06.279Z
2,stp-network,stpt,STP,https://assets.coingecko.com/coins/images/8713...,0.044790,87013801,252,8.701380e+07,1660226.0,0.047884,...,1.942420e+09,1.942420e+09,0.270085,-83.34417,2021-10-06T08:03:37.704Z,0.006467,595.56900,2019-10-01T00:00:00.000Z,"{'times': 3.47897548019818, 'currency': 'usd',...",2023-08-25T19:41:08.991Z
3,metars-genesis,mrs,Metars Genesis,https://assets.coingecko.com/coins/images/2662...,1.370000,86318551,253,1.373406e+09,47996.0,1.380000,...,1.000000e+09,NaN,4.300000,-68.03157,2022-08-06T23:23:06.656Z,0.584931,134.98834,2022-09-16T23:23:54.565Z,None,2023-08-25T19:39:46.959Z
4,playdapp,pla,PlayDapp,https://assets.coingecko.com/coins/images/1431...,0.153427,85953017,254,1.074166e+08,1205747.0,0.158928,...,7.000000e+08,NaN,3.740000,-95.88582,2021-10-31T01:43:56.805Z,0.093206,65.27843,2021-06-22T13:53:57.916Z,None,2023-08-25T19:41:15.065Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,moonriver,movr,Moonriver,https://assets.coingecko.com/coins/images/1798...,4.320000,32796817,496,4.723826e+07,1103317.0,4.380000,...,1.093687e+07,1.093675e+07,494.260000,-99.12256,2021-09-11T11:40:35.912Z,4.240000,2.36383,2023-08-25T14:16:02.397Z,None,2023-08-25T19:41:11.468Z
246,radio-caca,raca,Radio Caca,https://assets.coingecko.com/coins/images/1784...,0.000097,32682105,497,4.871171e+07,3111798.0,0.000100,...,4.301182e+11,5.000000e+11,0.011920,-99.18187,2021-11-14T09:16:57.279Z,0.000095,2.59169,2023-08-21T16:34:59.680Z,None,2023-08-25T19:41:09.874Z
247,aura-bal,aurabal,Aura BAL,https://assets.coingecko.com/coins/images/2653...,10.070000,32508255,498,3.250826e+07,152411.0,10.190000,...,3.225652e+06,NaN,21.610000,-53.26998,2023-01-31T20:55:03.507Z,8.200000,23.18122,2022-11-09T13:15:21.224Z,None,2023-08-25T19:40:37.757Z
248,singularitydao,sdao,SingularityDAO,https://assets.coingecko.com/coins/images/1538...,0.384596,32461370,501,3.846593e+07,817007.0,0.396259,...,1.000000e+08,1.000000e+08,6.620000,-94.19216,2021-09-06T23:53:52.687Z,0.145726,163.99055,2021-05-13T15:50:14.728Z,None,2023-08-25T19:41:16.296Z


In [127]:
df_global.describe()

,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,atl,atl_change_percentage
count,2.500000e+02,2.500000e+02,250.000000,2.390000e+02,2.500000e+02,2.470000e+02,2.470000e+02,247.000000,247.000000,2.480000e+02,248.000000,2.500000e+02,2.400000e+02,1.460000e+02,2.500000e+02,250.000000,250.000000,2.500000e+02
mean,5.393246e+02,4.260662e+09,125.500000,5.359994e+09,1.835339e+08,5.306494e+02,5.213682e+02,-0.010516,-0.510444,-5.138822e+05,-0.355974,6.364534e+14,1.764377e+15,2.893380e+15,5.510643e+05,-79.519256,129.387516,6.242558e+34
std,3.304454e+03,3.473525e+10,72.312977,3.802924e+10,1.203836e+09,3.343586e+03,3.288807e+03,9.794283,3.893129,5.715957e+07,4.054444,9.873927e+15,2.711019e+16,3.475827e+16,8.679827e+06,24.578779,881.144832,9.870351e+35
min,1.085000e-09,9.076841e+07,1.000000,9.318483e+07,0.000000e+00,1.115000e-09,1.049000e-09,-52.266161,-14.898320,-4.959962e+08,-14.976540,0.000000e+00,4.589498e+03,8.969497e+03,6.345000e-09,-100.000000,0.000000,1.940000e-01
25%,1.422795e-01,1.492023e+08,63.250000,2.033946e+08,2.005347e+06,1.495370e-01,1.412990e-01,-0.019180,-1.502815,-3.988402e+06,-1.475502,8.252326e+07,1.058640e+08,1.000000e+08,1.182500e+00,-95.701577,0.012162,4.464688e+01
50%,7.251155e-01,2.539952e+08,125.500000,4.475288e+08,8.573578e+06,7.728370e-01,7.086510e-01,-0.000346,-0.294780,-6.729346e+05,-0.250235,4.490943e+08,1.000000e+09,1.000000e+09,6.230000e+00,-91.024705,0.193030,2.821778e+02
75%,4.755000e+00,5.976754e+08,187.750000,1.042082e+09,2.688516e+07,4.835000e+00,4.640000e+00,0.000896,0.406035,1.435906e+06,0.439690,2.960389e+09,4.494373e+09,4.823742e+09,5.765000e+01,-71.476522,0.893429,1.840202e+03
max,2.600900e+04,5.055050e+11,250.000000,5.452941e+11,1.448336e+10,2.631700e+04,2.583100e+04,131.310000,30.100650,4.542439e+08,33.957640,1.561249e+17,4.200000e+17,4.200000e+17,1.372421e+08,-2.176990,10704.550000,1.560640e+37


In [128]:
df_global.columns

Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated'],
      dtype='object')

Comprobando la esactitud calculando el porcentaje de capitalizacion o dominancia de bitcoin con los datos provistos.

In [129]:
df_global[df_global['id'] == 'bitcoin']['market_cap'][0]/df_global['market_cap'].sum()

0.47457889334817804

Segun https://coin360.com/ los datos son correctos.

Eliminando columnas innecesarias

In [130]:
df_global.drop(['image','high_24h','ath_date','atl_date','low_24h','price_change_24h','price_change_percentage_24h','market_cap_change_24h','market_cap_change_percentage_24h','roi','ath_change_percentage','atl_change_percentage'],axis=1,inplace=True)

In [131]:
df_global.head()

,id,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,circulating_supply,total_supply,max_supply,ath,atl,last_updated
0,bitcoin,btc,Bitcoin,25981.000000,505505018618,1,5.452941e+11,1.045490e+10,1.946767e+07,2.100000e+07,2.100000e+07,69045.00,67.810000,2023-08-25T18:46:35.549Z
1,ethereum,eth,Ethereum,1648.470000,198056902645,2,1.980569e+11,5.191691e+09,1.202164e+08,1.202164e+08,NaN,4878.26,0.432979,2023-08-25T18:46:33.672Z
2,tether,usdt,Tether,0.999492,82805135256,3,8.280514e+10,1.448336e+10,8.284913e+10,8.284913e+10,NaN,1.32,0.572521,2023-08-25T18:45:01.333Z
3,binancecoin,bnb,BNB,217.700000,33510341897,4,4.356061e+10,3.866782e+08,1.538562e+08,1.538562e+08,2.000000e+08,686.31,0.039818,2023-08-25T18:46:41.477Z
4,ripple,xrp,XRP,0.523482,27705683768,5,5.235964e+10,8.715990e+08,5.291419e+10,9.998849e+10,1.000000e+11,3.40,0.002686,2023-08-25T18:46:37.917Z


In [132]:
df_global.rename(columns={'ath': 'max_price',
                   'atl': 'min_price',
                   }, inplace=True)

In [133]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       250 non-null    object 
 1   symbol                   250 non-null    object 
 2   name                     250 non-null    object 
 3   current_price            250 non-null    float64
 4   market_cap               250 non-null    int64  
 5   market_cap_rank          250 non-null    int64  
 6   fully_diluted_valuation  239 non-null    float64
 7   total_volume             250 non-null    float64
 8   circulating_supply       250 non-null    float64
 9   total_supply             240 non-null    float64
 10  max_supply               146 non-null    float64
 11  max_price                250 non-null    float64
 12  min_price                250 non-null    float64
 13  last_updated             250 non-null    object 
dtypes: float64(8), int64(2), o

El dataframe no contiene datos nulos o vacios.

In [134]:
df_global.to_csv('./Data/all_coins.csv',index=False)

***

Extrayendo informacion de exchanges

In [103]:
def get_exchange_info(limit=None, base_currency=None):
    base_url = "https://api.coingecko.com/api/v3/exchanges"
    params = {'per_page': 250}
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        exchanges_data = response.json()
        return exchanges_data
    else:
        print("Error:", response.status_code)
        return None

In [104]:
df_exchanges = get_exchange_info()

In [105]:
df_exchanges = pd.DataFrame(df_exchanges)
df_exchanges

,id,name,year_established,country,description,url,image,has_trading_incentive,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,binance,Binance,2017.0,Cayman Islands,,https://www.binance.com/,https://assets.coingecko.com/markets/images/52...,False,10.0,1.0,200455.723795,131112.818914
1,gdax,Coinbase Exchange,2012.0,United States,,https://coinbase-consumer.sjv.io/coingecko,https://assets.coingecko.com/markets/images/23...,False,10.0,2.0,30308.933223,30308.933223
2,bybit_spot,Bybit,2018.0,British Virgin Islands,Bybit is a cryptocurrency exchange that offers...,https://www.bybit.com,https://assets.coingecko.com/markets/images/69...,False,10.0,3.0,28728.843946,28728.843946
3,huobi,Huobi,2013.0,Seychelles,,https://www.huobi.com,https://assets.coingecko.com/markets/images/25...,False,10.0,4.0,38706.507138,19743.017826
4,kraken,Kraken,2011.0,United States,,https://r.kraken.com/c/2223866/687155/10583,https://assets.coingecko.com/markets/images/29...,False,10.0,5.0,18114.272692,13082.623548
...,...,...,...,...,...,...,...,...,...,...,...,...
245,verse,Verse,2022.0,None,"The Verse DEX, available at <a href=""http://ve...",https://verse.bitcoin.com/,https://assets.coingecko.com/markets/images/98...,False,5.0,245.0,0.342037,0.342037
246,polycat_finance,Polycat Finance,NaN,None,Polycat launched May 2021 as a yield farm and ...,https://polycat.finance/swap,https://assets.coingecko.com/markets/images/77...,False,5.0,246.0,0.304890,0.304890
247,fusionx-v2,FusionX V2,NaN,None,,https://fusionx.finance/swap?,https://assets.coingecko.com/markets/images/11...,False,5.0,247.0,0.273395,0.273395
248,spiritswap,SpiritSwap,2021.0,None,SpiritSwap is a decentralized exchange (DEX) o...,https://app.spiritswap.finance/#/,https://assets.coingecko.com/markets/images/67...,False,5.0,248.0,0.267439,0.267439


In [106]:
df_exchanges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               250 non-null    object 
 1   name                             250 non-null    object 
 2   year_established                 142 non-null    float64
 3   country                          129 non-null    object 
 4   description                      245 non-null    object 
 5   url                              250 non-null    object 
 6   image                            250 non-null    object 
 7   has_trading_incentive            247 non-null    object 
 8   trust_score                      249 non-null    float64
 9   trust_score_rank                 249 non-null    float64
 10  trade_volume_24h_btc             250 non-null    float64
 11  trade_volume_24h_btc_normalized  250 non-null    float64
dtypes: float64(5), object(

In [107]:
df_exchanges['has_trading_incentive'].unique()

array([False, None], dtype=object)

In [108]:
# Elimina las columnas con datos no relevantes
df_exchanges.drop(['url','image','description','has_trading_incentive'],axis=1,inplace=True)

In [109]:
df_exchanges[['trade_volume_24h_btc','trade_volume_24h_btc_normalized']] = df_exchanges[['trade_volume_24h_btc','trade_volume_24h_btc_normalized']].round(2)

In [110]:
df_exchanges[['year_established','trust_score_rank','trust_score']] = df_exchanges[['year_established','trust_score_rank','trust_score']].astype('Int64')

In [111]:
df_exchanges.head()

,id,name,year_established,country,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,binance,Binance,2017,Cayman Islands,10,1,200455.72,131112.82
1,gdax,Coinbase Exchange,2012,United States,10,2,30308.93,30308.93
2,bybit_spot,Bybit,2018,British Virgin Islands,10,3,28728.84,28728.84
3,huobi,Huobi,2013,Seychelles,10,4,38706.51,19743.02
4,kraken,Kraken,2011,United States,10,5,18114.27,13082.62


In [112]:
df_exchanges.to_csv('./Data/exchanges.csv',index=False)